# Business Understanding

## Business background

- Who is the client, what business domain the client is in.
  - Our client for this project is XYZ Realty, LLC. They're primarily
    a real estate company based in the west coast. They make profits
    by "flipping" houses, that is buying a house at a low value and
    selling at a higher value.
- What business problems are we trying to address?
  - XYZ Realty, LLC, is seeking to expand their portfolio, and are
    looking for areas in the United States with the best opportunity for
    them to make a profit. They have asked a pretty open ended question,
    what are the 5 best zip codes?

## Scope

- What data science solutions are we trying to build?
  - We plan to develop a set of forecasting models.
- What will we do?
  - We will develop a catalogue of forecasting models for the zip codes
    provided to us.
- How is it going to be consumed by the customer?
  - Our customer will have a report delivered to them, recommending the
    best zip codes to invest in, balancing both risk and profit.

## Metrics

- What are the qualitative objectives? (e.g. reduce user churn)
  - Develop models with low forecasting errors
- What is a quantifiable metric  (e.g. reduce the fraction of users with 4-week inactivity)
  - Our quantifiable metric is RMSE (model errors), and AIC (model complexity)
- Quantify what improvement in the values of the metrics are useful for the customer scenario (e.g. reduce the  fraction of users with 4-week inactivity by 20%) 
  - Reduce our baseline RMSE and AIC scores by 25%.
- What is the baseline (current) value of the metric? (e.g. current fraction of users with 4-week inactivity = 60%)
  - Current baseline value of the metric is +Infinity. This is a new project.
- How will we measure the metric? (e.g. A/B test on a specified subset for a specified period; or comparison of performance after implementation to baseline)
  - We'll measure the metric by doing walk-forward validation (RMSE), and by getting the AIC of our model.

## Plan

- Phases (milestones), timeline, short description of what we'll do in each phase.
  - Processing
  - EDA
  - Baseline Modeling
  - ARIMA
  - SARIMA

## Architecture

- Data
  - What data do we expect? Raw data in the customer data sources (e.g. on-prem files, SQL, on-prem Hadoop etc.)
    - We expect to receive flat files with the median market value of houses by zipcode over time.
- What tools and data storage/analytics resources will be used in the solution e.g.,
  - Python for feature construction, aggregation, and modeling.

# Data Mining

The data used for this project comes from the [Zillow Research Page](https://www.zillow.com/research/data/). We are using historical housing data which tells us the home value for houses between the 35th and 65th percentile in each zip code.

> A smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type.

The data can be downloaded from source over http by [clicking this link](http://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv). Due to the large nature of the dataset it has been stored in this repository as a compressed file. Since it is an immutable file it won't be subject to any changes, and won't throttle the cloning time.

# Data Cleaning

Our dataset for this project is stored relative to this notebook at `../data/raw/data.csv.xz`. Since pandas can infer the compression format, we can load this file similar to that of a regular flat file.

The main goals for our data cleaning are:

- Remove duplicates
- Impute/Remove missing data
- Transform data to more manageable format

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# load data and output head
df = pd.read_csv("../data/raw/data.csv.xz")
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


Looking at our data we can see that it is pivoted, with dates as column headers. We'll eventually have to unpivot this dataset into long-form for easier manipulation.

In [3]:
# output data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


Looking at the info output of our dataset it appears most of our data is correctly formatted, with only 4 columns typed as objects (State, City, Metro, CountyName).

In [4]:
# where are there missing values and how many missing values do we have
na = df.isna().sum()
na[na > 0]

Metro      1043
1996-04    1039
1996-05    1039
1996-06    1039
1996-07    1039
           ... 
2014-02      56
2014-03      56
2014-04      56
2014-05      56
2014-06      56
Length: 220, dtype: int64

We appear to have numerous missing values mainly in our date columns, but also in our Metro column. To fix that we could drop any rows with excessive missing values, and then backfill the remaining ones. For our Metro column, we could use some supervised ML to suggest appropriate labels, however it may be unlikely that the missing labels appear in our set of available labels.

## Missing Values

Our first task is decide whether to drop or impute the missing values in our Metro column.

In [5]:
# where are the missing values in our dataset
metro_missing = df.loc[df.Metro.isna(), :"SizeRank"]
metro_missing

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank
151,69340,27410,Greensboro,NC,NaN,Guilford,152
167,69336,27406,Greensboro,NC,NaN,Guilford,168
491,69270,27265,High Point,NC,NaN,Guilford,492
526,69337,27407,Greensboro,NC,NaN,Guilford,527
695,69335,27405,Greensboro,NC,NaN,Guilford,696
...,...,...,...,...,...,...,...
14684,70215,28775,Scaly Mountain,NC,NaN,Macon,14685
14694,99114,97149,Neskowin,OR,NaN,Tillamook,14695
14710,59210,3812,Bartlett,NH,NaN,Carroll,14711
14717,62697,12720,Bethel,NY,NaN,Sullivan,14718


In [6]:
# inspect similar rows
df[(df.CountyName == "Guilford") & (df.State == "NC")]

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
151,69340,27410,Greensboro,NC,NaN,Guilford,152,137100.0,136600.0,136000.0,...,212900,213200,213600,214300,215100,216200,217700,219600,221000,221500
167,69336,27406,Greensboro,NC,NaN,Guilford,168,80900.0,80700.0,80400.0,...,109700,107900,106700,107200,107900,108700,109500,110000,110700,111300
491,69270,27265,High Point,NC,NaN,Guilford,492,108000.0,107700.0,107400.0,...,159300,157400,156300,156600,156700,156600,156800,157000,157400,157800
526,69337,27407,Greensboro,NC,NaN,Guilford,527,93900.0,93700.0,93400.0,...,122000,120800,119800,119800,120300,121100,122200,123100,124000,124900
695,69335,27405,Greensboro,NC,NaN,Guilford,696,74100.0,73900.0,73800.0,...,91500,90200,89000,88700,88900,89700,91000,92100,93100,93700
3027,69354,27455,Greensboro,NC,NaN,Guilford,3028,111600.0,111500.0,111300.0,...,189800,190100,190000,189800,189900,190500,191400,192200,193100,193700
4476,69267,27262,High Point,NC,NaN,Guilford,4477,62700.0,62600.0,62600.0,...,105000,102700,101200,101300,101700,101700,101100,101000,102300,104400
5010,69339,27409,Greensboro,NC,NaN,Guilford,5011,101500.0,101500.0,101500.0,...,149300,149100,149500,149800,150000,150500,151200,151600,151900,152200
5109,69333,27403,Greensboro,NC,NaN,Guilford,5110,77000.0,76900.0,76700.0,...,132300,129100,126600,126000,126400,127600,129000,130000,130100,129800
5275,69331,27401,Greensboro,NC,NaN,Guilford,5276,62000.0,61800.0,61600.0,...,65700,65200,64800,64900,65500,66700,68500,70000,70600,70700


Looking at just a small subset of this data, we can already see that trying to label these missing values will be impossible with just our data alone. We'll opt to drop the column instead.

In [7]:
# drop metro column
df.drop(columns=["Metro"], inplace=True, errors="ignore")

Moving onto our date columns, we'll want to first start by dropping any rows with excessive amounts of missing values.

In [8]:
# drop any rows with more than 10% of the values missing
date_cols = df.loc[:, "1996-04":].columns
threshold = int(date_cols.size * .90)
df.dropna(thresh=threshold, subset=date_cols, inplace=True)
print("New Shape", df.shape)

New Shape (13687, 271)


In [9]:
# backfill rows missing data
bf_df = df.fillna(method="bfill", axis=1)
bf_df

,RegionID,RegionName,City,State,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Cook,1,334200,335400,336500,337600,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Collin,2,235700,236900,236700,235400,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Harris,3,210400,212200,212200,210700,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Cook,4,498100,500900,503100,504600,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,5,77300,77300,77300,77300,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Franklin,14719,94600,94300,94000,93700,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Grafton,14720,92700,92500,92400,92200,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Madison,14721,57100,57300,57500,57700,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,Gunnison,14722,191100,192400,193700,195000,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


Now that we have cleaned up our dataset we'll unpivot it.

In [10]:
# unpivot dataset
long_df = pd.melt(bf_df, bf_df.loc[:, :'SizeRank'].columns, var_name="Date")
long_df.head()

,RegionID,RegionName,City,State,CountyName,SizeRank,Date,value
0,84654,60657,Chicago,IL,Cook,1,1996-04,334200
1,90668,75070,McKinney,TX,Collin,2,1996-04,235700
2,91982,77494,Katy,TX,Harris,3,1996-04,210400
3,84616,60614,Chicago,IL,Cook,4,1996-04,498100
4,93144,79936,El Paso,TX,El Paso,5,1996-04,77300


In [11]:
# save the long form dataset
long_df.to_csv("../data/interim/data.csv.xz")